#### Loading Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from itertools import product

#Tests for sample comparison
from scipy.stats import spearmanr
from scipy.stats import shapiro
import scipy.stats as stats
from scipy.stats import mannwhitneyu

#ARIMA
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.stats.diagnostic import lilliefors
from statsmodels.tsa.seasonal import seasonal_decompose

#Prophet
from prophet import Prophet
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric
from prophet.diagnostics import cross_validation

#Libraries for tuning NHITS and envieroment setup
import logging
from ray import tune
from IPython.display import display
#NHITS
from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM, NHITS
from neuralforecast.auto import AutoNHITS, AutoLSTM
from neuralforecast.losses.pytorch import MAE,RMSE,MQLoss

from utilsforecast.plotting import plot_series
from utilsforecast.losses import mse, mae, rmse
from utilsforecast.evaluation import evaluate

#Model Evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder

#LightGBM
import lightgbm as lgb
from lightgbm import LGBMRegressor
from lightgbm.callback import early_stopping, log_evaluation


#### Loading Data

In [ ]:
train_set = pd.read_csv(r"/mnt/c/Users/aimeraj/Desktop/data/train.csv")
stores_set = pd.read_csv(r"/mnt/c/Users/aimeraj/Desktop/data/stores.csv")
oil_set = pd.read_csv(r"/mnt/c/Users/aimeraj/Desktop/data/oil.csv")
holiday_set = pd.read_csv(r"/mnt/c/Users/aimeraj/Desktop/data/holidays_events.csv")
transactions_set =pd.read_csv(r"/mnt/c/Users/aimeraj/Desktop/data/transactions.csv")

#### Display Options

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
#αν θελω reset
#pd.reset_option('display.max_columns')

#### Data Preparation

In [ ]:
#ελεγχος για διπλοτυπα
print(train_set.duplicated().sum(),
      stores_set.duplicated().sum(),
      oil_set.duplicated().sum(),
      holiday_set.duplicated().sum(),
      transactions_set.duplicated().sum()
      )

In [ ]:
train_set.head()

In [ ]:
stores_set.head()

In [ ]:
oil_set.head()

In [ ]:
holiday_set.head()

In [ ]:
holiday_set.locale.unique()

In [ ]:
transactions_set.head()

In [ ]:
#Merges
df = pd.merge(train_set,stores_set,on='store_nbr',how='left')
df = pd.merge(df,oil_set,on ='date',how='left')
df = pd.merge(df,transactions_set,on=['date','store_nbr'],how='left')

In [ ]:
df.head()

In [ ]:
print(
    f"The dataset contains {df.shape[0]} samples and "
    f"{df.shape[1]} features"
)

In [ ]:
#Κοιταω για Nas
df.isna().sum()

In [ ]:
sales_zero = df[df['sales'] == 0]

In [ ]:
sales_zero.isna().sum() # 291028 - 245869 = 45159

In [ ]:
df.loc[df['sales']==0,'transactions']=0
del sales_zero
df['transactions'] = df.groupby('store_nbr')['transactions'].ffill()

In [ ]:
#Απο αυτο βλεπουμε οτι δεν εχουμε την τιμη του πετρελαιου για ολες τις ημερομινιες
print(len(train_set['date'].unique()),len(oil_set['date'].unique()))

In [ ]:
df['date']= pd.to_datetime(df['date'])
df['day']= df['date'].dt.day_name()
working_days = df[df['date'].dt.dayofweek < 5]

In [ ]:
print(df.loc[df['day'] == 'Saturday', 'dcoilwtico'].isna().sum() +
      df.loc[df['day'] == 'Sunday', 'dcoilwtico'].isna().sum() 
      )

In [ ]:
#Βρισκω το Σ/Κ με τιμες Na στο πετρελαιο
weekend_mask = df['day'].isin(['Saturday', 'Sunday']) & df['dcoilwtico'].isna()
df['price_filled'] = df['dcoilwtico']
df['price_filled'] = df['price_filled'].where(df['day'].isin(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']))
df['price_filled'] = df['price_filled'].ffill()
df.loc[weekend_mask, 'dcoilwtico'] = df.loc[weekend_mask, 'price_filled']
df.drop(columns=['price_filled'], inplace=True)

In [ ]:
df['dcoilwtico'] = df['dcoilwtico'].interpolate(method='linear')
df['dcoilwtico'] = df['dcoilwtico'].bfill()
df['transactions'] = df['transactions'].fillna(0)

In [ ]:
#Για τις υπολοιπες τιμες του πετρελαιου που δεν εχουμε κανουμε imputation
#df['dcoilwtico'] = df['dcoilwtico'].ffill()

In [ ]:
holiday_set.isna().sum()

In [ ]:
df.info()

In [ ]:
transactions_set.isna().sum()

In [ ]:
df.isna().sum()

In [ ]:
df.isna().sum()

In [ ]:
df.head()

#### Exploratory Data Analysis

In [ ]:
df.head()

In [ ]:
print(
    f"The dataset contains {df.shape[0]} samples and "
    f"{df.shape[1]} features"
)

In [ ]:
len(df['store_nbr'].unique())

In [ ]:
df['family'].unique()

In [ ]:
len(df['family'].unique())

In [ ]:
df['city'].unique()

In [ ]:
len(df['city'].unique())

In [ ]:
df['state'].unique()

In [ ]:
len(df['state'].unique())

In [ ]:
df['type'].unique()

In [ ]:
df['cluster'].unique()

In [ ]:
df['day'].unique()

In [ ]:
df[['sales','onpromotion','transactions','dcoilwtico']].describe()

In [ ]:
#Εδω δημιουργω ενα γραφημα για τα sales σε βαθος χρονου
#Για να γινει αυτο θα πρεπει να αθροισω τα sales καθε μερας καθως το υπαρχον dataset εχει 
#sales για ολες τις ημερες 
daily_sales = df[['date','sales']].copy()
daily_sales = daily_sales.groupby('date').sum().reset_index()


In [ ]:
px.line(daily_sales,x='date',y='sales',title="Daily Sales")

In [ ]:
#Εδω ενονω τις τιμες του πετρελαιου με τα daily sales για να κανω correlation test
daily_oil = df[['date', 'dcoilwtico']].dropna().drop_duplicates(subset='date')
daily_df = pd.merge(daily_sales,daily_oil,on='date',how='left')
daily_df.head()

In [ ]:
moving_average = daily_df['sales'].rolling(
    window = len(daily_df['sales']),
    center = True,
    min_periods = len(daily_df['sales'])//2,
).mean()

daily_df['moving_average'] = moving_average

In [ ]:
fig1 = px.scatter(daily_df, x='date', y='sales')
fig2 = px.line(daily_df, x='date', y='moving_average', color_discrete_sequence=['red'])
fig3 = go.Figure(data=fig1.data + fig2.data)
fig3 = fig3.update_layout(xaxis_title="Sales", yaxis_title="Date", title = 'Moving Average Over Time')
fig3.show()

In [ ]:
df['year'] = df['date'].dt.year
avg_sales_year_family = df.groupby(['year', 'family'])['sales'].mean().reset_index()

In [ ]:

fig = px.line(
    avg_sales_year_family,
    x='year',
    y='sales',
    color='family',
    facet_col='family',
    facet_col_wrap= 4,
    markers=True,
    title='Average Yearly Sales per Product Family'
)
fig.update_yaxes(matches=None)
fig.update_layout(height=2000,showlegend=False)

# fig.update_layout(
#     xaxis_title='Year',
#     yaxis_title='Average Sales',
#     legend_title='Family',
#     width=1000,
#     height=600
# )

fig.show()


In [ ]:
df['month'] = df['date'].dt.month
df['month_name'] = df['date'].dt.strftime('%B')
avg_sales_by_month = df.groupby(['month', 'month_name', 'family'])['sales'].mean().reset_index()

avg_sales_by_month = avg_sales_by_month.sort_values('month')

In [ ]:
fig = px.line(
    avg_sales_by_month,
    x='month_name',
    y='sales',
    color='family',
    facet_col='family',
    facet_col_spacing=0.1,
    facet_col_wrap=4,
    markers=True,
    category_orders={"month_name": [
        "January", "February", "March", "April", "May", "June",
        "July", "August", "September", "October", "November", "December"
    ]},
    title='Average Sales by Calendar Month per Product Family'
)
fig.update_yaxes(matches=None)
fig.update_layout(height=2000,showlegend=False)

# fig.update_layout(
#     xaxis_title='Month',
#     yaxis_title='Average Sales',
#     legend_title='Family',
#     width=1000,
#     height=600
# )

fig.show()


In [ ]:
df['day_nbr'] = df['date'].dt.weekday
avg_sales_by_weekday = df.groupby(['day_nbr', 'day', 'family'])['sales'].mean().reset_index()

In [ ]:
avg_sales_by_weekday = avg_sales_by_weekday.sort_values('day_nbr')
fig = px.line(
    avg_sales_by_weekday,
    x='day',
    y='sales',
    color='family',
    facet_col='family',
    facet_col_wrap=4,
    markers=True,
    category_orders={"day": [
        "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"
    ]},
    title='Average Sales by Day of Week per Product Family'
)

fig.update_yaxes(matches=None)
fig.update_layout(height=2000,showlegend=False)

# fig.update_layout(
#     xaxis_title='Day of Week',
#     yaxis_title='Average Sales',
#     legend_title='Family',
#     width=1000,
#     height=600
# )

fig.show()


In [ ]:
t = df.groupby("family")['sales'].mean().sort_values(ascending=False).reset_index()
px.bar(t,y='family',x='sales',color='family',title='Average sales per product family')

In [ ]:
t = df.groupby("city")['sales'].mean().sort_values(ascending=False).reset_index()
px.bar(t,y='city',x='sales',color='city',title='Average sales per City')

In [ ]:

temp = df[["store_nbr", "sales"]].copy()
temp["ind"] = 1
temp["ind"] = temp.groupby("store_nbr")["ind"].cumsum().values
pivot_sales = pd.pivot(temp, index="ind", columns="store_nbr", values="sales")

corr_matrix = pivot_sales.corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

plt.figure(figsize=(20, 20))
sns.heatmap(corr_matrix,
            annot=True,
            fmt='.1f',
            cmap='coolwarm',
            square=True,
            mask=mask,
            linewidths=1,
            cbar=False)
plt.title("Correlations among stores", fontsize=20)
plt.show()

#### Oil Price - Impact on Sales

In [ ]:
#Αυτο εχει και αλλη δουλεια
px.line(daily_oil,x='date',y='dcoilwtico')

In [ ]:
plt.hist(daily_df['sales'],bins=30, color='skyblue', edgecolor='black');

In [ ]:
plt.hist(daily_df['dcoilwtico'],bins=30, color='skyblue', edgecolor='black');

In [ ]:
corr, p_value = spearmanr(daily_df['sales'], daily_df['dcoilwtico'])

print(f"Spearman correlation: {corr}")
print(f"P-value: {p_value}")

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(daily_df['dcoilwtico'], daily_df['sales'], alpha=0.5)
plt.title('Sales ~ Dcoilwtico')
plt.xlabel('dcoilwtico')
plt.ylabel('sales')
plt.grid(True)
plt.show()


#### OnPromotion - Impact on Sales

In [ ]:
corr, p_value = spearmanr(df['sales'], df['onpromotion'])

print(f"Spearman correlation: {corr}")
print(f"P-value: {p_value}")


In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df['onpromotion'],df['sales'], alpha=0.5)
plt.title('Sales ~ Onpromotion')
plt.xlabel('Onpromotion')
plt.ylabel('sales')
plt.grid(True)
plt.show()

#### Earthquake - Impact on Sales

In [ ]:
daily_df["date"] = pd.to_datetime(daily_df["date"])
before = daily_df[(daily_df["date"] >= '2016-03-01') & (daily_df["date"] <= '2016-04-15')]
after = daily_df[(daily_df["date"] >= '2016-04-16') & (daily_df["date"] <= '2016-05-31')]

In [ ]:
plt.hist(before['sales'], bins=30, color='skyblue', edgecolor='black');


In [ ]:
plt.hist(after['sales'], bins=30, color='skyblue', edgecolor='black');


In [ ]:
stat, p_value = mannwhitneyu(before['sales'], after['sales'])

print(f"U statistic: {stat}")
print(f"P-value: {p_value}")

In [ ]:
before[['sales']].describe()

In [ ]:
after[['sales']].describe()

In [ ]:
df[['sales']].describe()

In [ ]:
del avg_sales_by_month,avg_sales_by_weekday,avg_sales_year_family,moving_average,daily_df,daily_oil,daily_sales
del train_set,oil_set,transactions_set,stores_set,before,after,weekend_mask
del fig,fig1,fig2,fig3,working_days,

In [ ]:
%who

In [ ]:
holiday_set['date'] = pd.to_datetime(holiday_set['date'])
holiday_set[holiday_set['transferred']==True]

In [ ]:
#Το προβλημα με τα transfers
temp1 = holiday_set[(holiday_set['type']=='Holiday')& (holiday_set['transferred']==True)].drop('transferred',axis=1).reset_index(drop=True)
temp2 = holiday_set[(holiday_set['type']=='Transfer')].drop('transferred',axis=1).reset_index(drop=True)
temp = pd.concat([temp1,temp2],axis=1)
temp = temp.iloc[:,[5,1,2,3,4,]]
holiday_set = holiday_set[(holiday_set['type']!='Transfer')& (holiday_set['transferred']==False)].drop('transferred',axis=1)
holiday_set = pd.concat([holiday_set, temp]).reset_index(drop=True)
del temp,temp1,temp2

In [ ]:
holiday_set['type']= np.where(holiday_set['type']=='Additional','Holiday',holiday_set['type'])
holiday_set['type']= np.where(holiday_set['type']=='Bridge','Holiday',holiday_set['type'])

In [ ]:
work_days_left = holiday_set[holiday_set.type=='Work Day']
holiday_set = holiday_set[holiday_set.type!='Work Day']

In [ ]:
#Κραταω αυτο για το Prophet μετα
holidays_for_prophet = holiday_set.drop(['type','locale','locale_name'],axis=1)

In [ ]:
events = holiday_set[holiday_set.type=='Event'].drop(['type','locale','locale_name'],axis=1)
events= events.rename({"description":"event"},axis=1)
holiday_set = holiday_set[holiday_set.type!='Event'].drop("type",axis=1)

In [ ]:
regional = holiday_set[holiday_set['locale']=='Regional'].rename({"locale_name":"state","description":"holiday_regional"},axis=1).drop("locale",axis=1).drop_duplicates()
national = holiday_set[holiday_set['locale']=='National'].rename({"description":"holiday_national"},axis=1).drop(["locale","locale_name"],axis=1).drop_duplicates()
local = holiday_set[holiday_set['locale']=='Local'].rename({"locale_name":"city","description":"holiday_local"},axis=1).drop("locale",axis=1).drop_duplicates()

In [ ]:
national['date'] = pd.to_datetime(national['date'])
regional['date'] = pd.to_datetime(regional['date'])
df = pd.merge(df,national,how='left',on='date')
df = pd.merge(df,regional,how='left',on=['date',"state"])
df = pd.merge(df,local,how='left',on=['date',"city"])
df = pd.merge(df,events,on=['date'],how='left')
del national,regional,local

In [ ]:
df["holiday_national_binary"] = np.where(df["holiday_national"].notnull(),1,0)
df["holiday_regional_binary"] = np.where(df["holiday_regional"].notnull(),1,0)
df["holiday_local_binary"] = np.where(df["holiday_local"].notnull(),1,0)

In [ ]:
encoded_events= pd.get_dummies(events,"event",dtype=int)
encoded_events.columns=encoded_events.columns.str.replace(" ","_")
encoded_events.columns=encoded_events.columns.str.replace(":","_")
encoded_events.columns=encoded_events.columns.str.replace("+","_")
encoded_events.columns=encoded_events.columns.str.replace("-","_")
df = pd.merge(df,encoded_events,how='left',on='date')
del encoded_events,events

In [ ]:
df['event'] = df['event'].fillna('no_event')
df['holiday_national'] = df['holiday_national'].fillna('no_holiday')
df['holiday_regional'] = df['holiday_regional'].fillna('no_holiday')
df['holiday_local'] = df['holiday_local'].fillna('no_holiday')
event_cols = [col for col in df.columns if col.startswith('event')]
event_cols.remove('event')
df[event_cols] = df[event_cols].fillna(0).astype(int)

In [ ]:
mylist = ['holiday_national_binary','holiday_regional_binary','holiday_local_binary']
mylist.append(event_cols)

In [ ]:
df.shape

In [ ]:
def AB_Test(d,group,target):
    groupA = d[d[group] == 1][target]
    groupB = d[d[group] == 0][target]
    if(len(groupA)<50):
        t1 = shapiro(groupA)[1] < 0.05
    else:
        t1 = lilliefors(groupA)[1] < 0.05
    
    if(len(groupB)<50):
        t2 = shapiro(groupB)[1] < 0.05
    else:
        t2 = lilliefors(groupB)[1] < 0.05
    
    if (t1 == False) & (t1 == False):
        leveneTest = stats.levene(groupA, groupB)[1] < 0.05
        
        if leveneTest == False:
            ttest = stats.ttest_ind(groupA, groupB, equal_var=True)[1]

        else:
            ttest = stats.wilcoxon(groupA, groupB, equal_var=False)[1]

    else:
        ttest = stats.mannwhitneyu(groupA, groupB)[1] 

    temp = pd.DataFrame({
        "variable":[group],
        "p-value":[ttest]
    })
    return temp

results = []

for i in ['holiday_national_binary','holiday_regional_binary','holiday_local_binary']:
    results.append(AB_Test(d=df[df['sales'].notnull()],group=i,target='sales'))

results = pd.concat(results)

In [ ]:
results

In [ ]:
results=[]
for i in event_cols:
    results.append(AB_Test(d=df[df['sales'].notnull()],group=i,target='sales'))
results = pd.concat(results)
results

In [ ]:
results.loc[results['p-value']>0.05,]
del results

In [ ]:
df['unique_id'] = df['store_nbr'].astype(str) + '_' + df['family']


In [ ]:
#Zero forecasting
zero_sales = df[['unique_id','sales']].copy()
zero_sales=zero_sales.groupby(['unique_id']).sum().reset_index()
zero_sales=zero_sales[zero_sales['sales']==0]

In [ ]:
zero_sales

In [ ]:
zero_sales.shape # βαση αυτου υπαρχουν 53 combos που εχουν 0 πωλησεις

#### Models

In [ ]:
#Prophet

In [ ]:
agg_df = df.groupby(['date','unique_id']).agg({'sales':'sum'}).reset_index().sort_values(['unique_id','date'])
total_sales_df = agg_df.pivot(index='date',columns='unique_id', values='sales')

In [ ]:
changepoint_prior_scale_range = np.linspace(0.001, 0.5, num=5).tolist()
print(changepoint_prior_scale_range)

In [ ]:
seasonality_prior_scale_range = np.linspace(0.01, 10, num=5).tolist()
holidays_prior_scale_range = np.linspace(0.01, 10, num=5).tolist()

In [ ]:
dicts = {}

for feature in total_sales_df.columns[:1]:
  
    category_df = total_sales_df[feature].copy().reset_index()
    category_df.columns = ["ds", "y"]

    category_df[["y"]] = category_df[["y"]].apply(pd.to_numeric)
    category_df["ds"] = pd.to_datetime(category_df["ds"])
    
    param_grid = {  
        "changepoint_prior_scale": changepoint_prior_scale_range,
        "seasonality_prior_scale": seasonality_prior_scale_range }

    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in product(*param_grid.values())]
    maes = [] 

    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params).fit(category_df)  # Fit model with given params
        df_cv = cross_validation(m, initial="365 days", period="30 days", horizon = "30 days")
        df_p = performance_metrics(df_cv, rolling_window=1)
        print(df_p)
        maes.append(df_p["mae"].values[0])

    # Find the best parameters
    tuning_results = pd.DataFrame(all_params)
    tuning_results["mae"] = maes
    
    print(feature)
    print(tuning_results.head())

    params_dict = dict(tuning_results.sort_values("mae").reset_index(drop=True).iloc[0])
    params_dict["column"] = feature 
    
    dicts[feature] = params_dict

In [ ]:
prediction_days = 30
forecast_start_date = '2017-07-15'

In [ ]:
holidays_for_prophet=holidays_for_prophet.rename({'date':'ds','description':'holiday'},axis=1)
holidays_for_prophet['ds'] = pd.to_datetime(holidays_for_prophet['ds'])

In [ ]:
forecasted_dfs = []

for feature in total_sales_df.columns[:1]:

    df_copy = total_sales_df[feature].copy().reset_index()
    df_copy.columns = ['ds','y']
    df_copy[['y']] = df_copy[['y']].apply(pd.to_numeric)

    df_copy['ds'] = pd.to_datetime(df_copy['ds'])
    
    df_copy_ = df_copy[df_copy['ds'] < forecast_start_date]
    
    #finding the right params_dict for this unique_id
    params_dict = dicts[feature]
    
    #model
    m = Prophet(changepoint_prior_scale = dicts[feature]['changepoint_prior_scale'],
                seasonality_prior_scale = dicts[feature]['seasonality_prior_scale'],
                seasonality_mode = 'multiplicative',
                holidays=holidays_for_prophet
                )
    
    m.fit(df_copy_)

    future = m.make_future_dataframe(periods=prediction_days)
    fcst_prophet_train = m.predict(future)
    
    filter = fcst_prophet_train['ds']>=forecast_start_date 
    predicted_df = fcst_prophet_train[filter][['ds','yhat']]
    predicted_df = predicted_df.merge(df_copy)

    print(feature,mean_squared_log_error(predicted_df['y'],predicted_df['yhat']))

In [ ]:
forecasted_dfs = []

for feature in total_sales_df.columns[:1]:
    
    #formatting
    df_copy = total_sales_df[feature].copy().reset_index()
    df_copy.columns = ['ds','y']
    df_copy[['y']] = df_copy[['y']].apply(pd.to_numeric)

    df_copy['ds'] = pd.to_datetime(df_copy['ds'])
    
    df_copy_ = df_copy[df_copy['ds'] < forecast_start_date]
    
    #finding the right params_dict for this category
    params_dict = dicts[feature]
    
    #model
    m = Prophet(changepoint_prior_scale = dicts[feature]['changepoint_prior_scale'],
                seasonality_prior_scale = dicts[feature]['seasonality_prior_scale'],
                seasonality_mode = 'multiplicative',
                holidays=holidays_for_prophet
                )
    
    m.fit(df_copy_)

    future = m.make_future_dataframe(periods=prediction_days)
    fcst_prophet_train = m.predict(future)
    
    fig1 = m.plot(fcst_prophet_train)
    fig2 = m.plot_components(fcst_prophet_train)

    forecasted_df = fcst_prophet_train[fcst_prophet_train['ds']>=forecast_start_date]
    
    #forecasted_dfs.append(forecasted_df) #οταν αλλαξω το ποσα combos θα δω τοτε θα βαλω αυτο

In [ ]:
original = df.copy()

In [ ]:
%who

In [ ]:
#LightGBM

In [ ]:
for col in ['family', 'city', 'state', 'type', 'cluster']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [ ]:
y = df['sales']
features = ['family','onpromotion',
 'day_nbr',
 'store_nbr',
 'year',
 'month',
 'city',
 'state',
 'type',
 'cluster',
 'holiday_national_binary']
X = df[features]

In [ ]:
# Drop duplicates
# X_train = X_train.loc[:, ~X_train.columns.duplicated()]
# X_val = X_val.loc[:, ~X_val.columns.duplicated()]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val)

model = LGBMRegressor(
    objective='tweedie',
    metric='rmse',                   
    boosting_type='gbdt',            
    num_leaves=64,                   
    max_depth=7,                     
    learning_rate=0.05,              
    n_estimators=1000,               
    subsample=0.8,                   
    colsample_bytree=0.8,            
    reg_alpha=0.1,                   
    reg_lambda=0.1,                  
    random_state=1
)

# Fit with callbacks
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    callbacks=[early_stopping(stopping_rounds=50), log_evaluation(period=100)]
)

y_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))

In [ ]:
y_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))

In [ ]:
rmses = []
tscv = TimeSeriesSplit(n_splits=5)
for train_idx, val_idx in tscv.split(X):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        callbacks=[early_stopping(stopping_rounds=50), log_evaluation(period=100)])

    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    rmses.append(rmse)

In [ ]:
# Feature importance (split count)
importance_df = pd.DataFrame({
    'feature': model.feature_name_,
    'importance': model.feature_importances_
}).sort_values(by='importance', ascending=True)

# Horizontal bar plot
plt.figure(figsize=(10, 6))
plt.barh(importance_df['feature'], importance_df['importance'])
plt.xlabel('Split Count')
plt.title('Feature Importance')
plt.tight_layout()
plt.show()

In [ ]:
model.booster_.dump_model()['tree_info'][0]  # Δείχνει το πρώτο δέντρο


In [ ]:
# Δείξε το πρώτο δέντρο
ax = lgb.plot_tree(model, tree_index=0, figsize=(30, 10), show_info=['split_gain', 'internal_value', 'internal_count', 'leaf_count'])
plt.show()

In [ ]:
print(np.mean(rmses))

In [ ]:
importance_df = pd.DataFrame({
    'feature': features,
    'importance': model.feature_importances_
})

# Sort by importance
importance_df = importance_df.sort_values(by='importance', ascending=False)

# Plot
plt.figure(figsize=(20, 16))
plt.barh(importance_df['feature'], importance_df['importance'])
plt.xlabel('Importance Score')
plt.title('Feature Importance (Split Count)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
fig = px.line(
    results,
    x='date',
    y=['sales', 'predicted_sales'],
    title='Actual vs Predicted Sales (XGBoost)',
    labels={'value': 'Sales', 'variable': 'Legend'}
)
fig.show()

In [ ]:
df = original.copy()

In [ ]:
#NHITS

# Θελω για αρχη να τρεξω το μοντελο μονο με τις static μεταβλητες
df = df.drop([ 'id','onpromotion', 'city','state', 'type', 'cluster', 'dcoilwtico', 'transactions', 'day','holiday_national', 'holiday_regional', 'holiday_local', 'event',
       'holiday_national_binary', 'holiday_regional_binary',
       'holiday_local_binary',],axis=1)
df['unique_id'] = df['store_nbr'].astype(str) + '_' + df['family']
df['ds'] = pd.to_datetime(df['date'])
df['y'] = df['sales']

In [ ]:
logging.getLogger("pytorch_lightning").setLevel(logging.INFO)

In [ ]:
duplicates = df.groupby(['unique_id', 'ds']).size().reset_index(name='count')
duplicates = duplicates[duplicates['count'] > 1]

if duplicates.empty:
    print("✅ Κάθε unique_id εμφανίζεται το πολύ μία φορά ανά ημερομηνία.")
else:
    print("❌ Υπάρχουν διπλότυπα:")
    display(duplicates.head())

df_agg = df.groupby(['unique_id', 'ds'], as_index=False)['y'].sum()
df = df_agg

# Split into train and future (NHITS uses sliding windows, not test sets in the usual sense)
df_train = df.groupby('unique_id').apply(lambda x: x.iloc[:-30]).reset_index(drop=True)
df_test = df.groupby('unique_id').apply(lambda x: x.iloc[-30:]).reset_index(drop=True)


model = NeuralForecast(
    models=[NHITS(input_size=90, h=30, max_steps=2000, scaler_type='standard', loss=RMSE(),learning_rate=5e-5)],
    freq='D',
)

# model = NeuralForecast(models=[NHITS(input_size=2*H, h=H)], freq='D')
# Fit the model
model.fit(df_train)

In [ ]:
# Predict
df_forecast = model.predict()

# Rename 'y' to 'y_actual' in df_test to match expected column name after merge
df_test_renamed = df_test.rename(columns={'y': 'y_actual'})

In [ ]:
# Merge forecasts and actuals
df_eval = pd.merge(
    df_forecast, 
    df_test_renamed, 
    on=['unique_id', 'ds'], 
    how='left'
)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error,mean_squared_log_error
import numpy as np
# Calculate metrics
mae = mean_absolute_error(df_eval['y_actual'], df_eval['NHITS'])
rmse = np.sqrt(mean_squared_error(df_eval['y_actual'], df_eval['NHITS']))
#rmsle = np.sqrt(mean_squared_log_error(df_eval['y_actual'], df_eval['NHITS']))
print(f"MAE:  {mae:.2f}")
print(f"RMSE:  {rmse:.2f}")
#print(f"RMSLE:  {rmsle:.2f}")

In [ ]:
df_forecast= df_forecast.reset_index()

In [ ]:
plot_series(df,df_forecast, max_insample_length=24*5)

In [ ]:
plot_series(df[df['unique_id'].isin(['10_AUTOMOTIVE', '2_BEVERAGES'])])

In [ ]:
# # Ensure both components are strings
# df['store_nbr'] = df['store_nbr'].astype(str)
# df['family'] = df['family'].astype(str)

# # Create the unique_id column
# df['unique_id'] = df['store_nbr'] + '_' + df['family']

In [ ]:
# df = df.rename(columns={'date': 'ds'})
# df_forecast = df[['ds', 'sales', 'unique_id']].rename(columns={'sales': 'y'})
# df_forecast = df_forecast.set_index('unique_id')

In [ ]:
# df_2017 = df_forecast[df_forecast['ds'].dt.year == 2017]

In [ ]:
# df_2017 = df_2017.reset_index()
# df_2017.head()


In [ ]:
# df_2017.to_csv(r"/mnt/c/Users/aimeraj/Desktop/data2/df_2017.csv",index=False)

In [ ]:
# static_df = static_df.set_index('unique_id')
# static_df =static_df.drop('Unnamed: 0',axis=1)

In [ ]:
# config_nhits = {
#     "input_size": tune.choice([48, 96, 144]),            
#     "start_padding_enabled": True,
#     "n_blocks": [1, 1, 1, 1, 1],                         
#     "mlp_units": [[64, 64]] * 5,                         
#     "n_pool_kernel_size": tune.choice([
#         [1, 1, 1, 1, 1],
#         [2, 2, 2, 2, 2],
#         [4, 4, 4, 4, 4],
#         [8, 4, 2, 1, 1]
#     ]),
#     "n_freq_downsample": tune.choice([
#         [8, 4, 2, 1, 1],
#         [1, 1, 1, 1, 1]
#     ]),
#     "learning_rate": tune.loguniform(1e-4, 1e-2),
#     "scaler_type": None,
#     "max_steps": 1000,
#     "batch_size": tune.choice([1, 4, 10]),
#     "windows_batch_size": tune.choice([128]),
#     "random_seed": tune.randint(1, 21),                   
#}

In [ ]:
# horizon = 15
# model = [
#     NHITS(
#         h=horizon,
#         input_size= 10 * horizon,        # Use 75 past days (tune this!)
#         max_steps=100,                   # Total training steps (tune based on dataset size)
#         stat_exog_list=static_df.columns.tolist(),  # List of static variable names
#         scaler_type='robust',            # Robust scaling handles outliers better
#         loss=MQLoss(),                   # Quantile loss (good for uncertainty)
#         n_blocks=[1, 1, 1],              # Simplify if training is slow
#     )
# ]

In [ ]:
### Διαφορα πραγματα που ειχα γραψει προσπαθωντας να τρεξω NHITS 
### Τα εκανα comment out επειδη δεν ειναι με σειρα ωστε να δουλευουν

# df_2017.head()
# df_2017 = df_2017.loc["1_AUTOMOTIVE"]
# static_df = static_df[['1_AUTOMOTIVE']]
# nf = NeuralForecast(models=model, freq='D')
# nf.fit(df=df_2017, static_df=static_df)
# Y_hat_df = nf.predict()
# # Όταν κάνεις reset_index, βάλε το σωστό όνομα:
# df_2017_reset = df_2017.reset_index().rename(columns={'index': 'unique_id'})
# df_2017 = df_2017.reset_index()
# # Βήμα 1: Reset το df σου
# df_2017_reset = df_2017.reset_index()

# # Βήμα 2: Reset και το forecast
# Y_hat_df = Y_hat_df.reset_index()

# # Βήμα 3: Τσέκαρε τι στήλες έχουν
# print(df_2017_reset.columns)
# print(Y_hat_df.columns)

# # Βήμα 4: Plot
# plot_series(df_2017_reset, Y_hat_df, max_insample_length=24*5)

# df_2017_reset = df_2017.reset_index()  # φέρνει το unique_id ως στήλη
# assert set(df_2017.index.unique()) <= set(static_df.index), "Mismatch in unique_ids!"

# missing_ids = set(df_2017.index.unique()) - set(static_df.index)
# print(missing_ids)

# for missing_id in missing_ids:
#     static_df.loc[missing_id] = [0] * static_df.shape[1]  # ή βάλε default τιμές

# nf = NeuralForecast(models=model, freq='D')  # Daily frequency
# nf.fit(df= df_2017, static_df=static_df)

# df_forecast.head()
# df_2017 = df_forecast[df_forecast['ds'].dt.year == 2017]
# del df_forecast

# nf = NeuralForecast(models=model, freq='D')
#nf.fit(df=df_2017, static_df=static_df)